In [9]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import cv2

from PIL import Image
import numpy as np
import sys
sys.path.append('C:\Program Files (x86)\IntelSWTools\openvino_2019.2.275\python\python3.6')

In [61]:
pb_file = './model/tf_model.pb'
output_dir = './model'
img_height = 28
input_shape = [1, img_height,img_height,1]
input_shape_str = str(input_shape).replace(' ','')
input_shape_str

'[1,28,28,1]'

In [62]:
mo_tf_path = '"C:\Program Files (x86)\IntelSWTools\openvino\deployment_tools\model_optimizer\mo_tf.py"'

In [63]:
%run -i {mo_tf_path} --input_model {pb_file} --output_dir {output_dir} --input_shape {input_shape_str} --data_type FP16

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	D:\Workspace\GitHub\ISS-VSE-2019-09-23-IS1FT-CA2-Part1\./model/tf_model.pb
	- Path for generated IR: 	D:\Workspace\GitHub\ISS-VSE-2019-09-23-IS1FT-CA2-Part1\./model
	- IR output name: 	tf_model
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1,28,28,1]
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	False
TensorFlow specific parameters:
	- Input model in text protobuf format: 	False
	- Path to model dump for TensorBoard: 	None
	- List of shared libraries with TensorFlow custom layers implementation: 	None
	- Update the configuration file w

In [64]:
try:
    from openvino import inference_engine as ie
    from openvino.inference_engine import IENetwork, IEPlugin
except Exception as e:
    exception_type = type(e).__name__
    print("The following error happened while importing Python API module:\n[ {} ] {}".format(exception_type, e))

In [65]:
# Plugin initialization for specified device and load extensions library if specified.
plugin_dir = None
model_xml = './model/tf_model.xml'
model_bin = './model/tf_model.bin'
# Devices: GPU (intel), CPU, MYRIAD
plugin = IEPlugin("GPU", plugin_dirs=plugin_dir)
# Read IR
net = IENetwork(model=model_xml, weights=model_bin)
assert len(net.inputs.keys()) == 1
assert len(net.outputs) == 1
input_blob = next(iter(net.inputs))
out_blob = next(iter(net.outputs))
# Load network to the plugin
exec_net = plugin.load(network=net)
del net

In [48]:
def pre_process_image(imagePath, img_height=28):
    # Model input format
    n, c, h, w = [1, 1, img_height, img_height]
    image = Image.open(imagePath)
    processedImg = image.resize((h, w), resample=Image.BILINEAR)

    # Normalize to keep data between 0 - 1
    processedImg = (np.array(processedImg) - 0) / 255.0

    # Change data layout from HWC to CHW
    processedImg = processedImg.transpose((2, 0, 1))
    processedImg = processedImg.reshape((n, c, h, w))

    return image, processedImg, imagePath

In [69]:
image = Image.open('unnamed-chunk-60-1.png')

AttributeError: 'PngImageFile' object has no attribute 'shape'

In [66]:
# Run inference
fileName = 'unnamed-chunk-60-1.png'
# image = Image.open(imagePath)
image, processedImg, imagePath = pre_process_image(fileName)
exec_net.start_async(request_id=0, inputs={input_blob: processedImg})
res = exec_net.infer(inputs={input_blob: processedImg})
# Access the results and get the index of the highest confidence score
output_node_name = list(res.keys())[0]
res = res[output_node_name]
# idx = np.argsort(res[0])[-1]
# idx

ValueError: cannot reshape array of size 3136 into shape (1,3,28,28)

In [25]:
cap = cv2.VideoCapture(0)

In [29]:
while cap.isOpened():
    ret, frame = cap.read()
    in_frame = cv2.resize(frame, (416, 416))
    in_frame = in_frame.transpose((2, 0, 1))  # Change data layout from HWC to CHW
    in_frame = in_frame.reshape((1, 3, 416, 416))
    exec_net.start_async(request_id=0, inputs={input_blob: in_frame})
    objects = list()
    if exec_net.requests[0].wait(-1) == 0:
        output = exec_net.requests[0].outputs
        for layer_name, out_blob in output.items():
            layer_params = YoloV3Params(exec_net.layers[layer_name].params, out_blob.shape[2])
            log.info("Layer {} parameters: ".format(layer_name))
            layer_params.log_params()
            print(out_blob.shape)
            objects += parse_yolo_region(out_blob, in_frame.shape[2:],
                                         frame.shape[:-1], layer_params,
                                         args.prob_threshold)
    objects = sorted(objects, key=lambda obj : obj['confidence'], reverse=True)
    for i in range(len(objects)):
        if objects[i]['confidence'] == 0:
            continue
        for j in range(i + 1, len(objects)):
            if intersection_over_union(objects[i], objects[j]) > args.iou_threshold:
                objects[j]['confidence'] = 0

    # Drawing objects with respect to the --prob_threshold CLI parameter
    objects = [obj for obj in objects if obj['confidence'] >= args.prob_threshold]

    if len(objects) and args.raw_output_message:
        log.info("\nDetected boxes for batch {}:".format(1))
        log.info(" Class ID | Confidence | XMIN | YMIN | XMAX | YMAX | COLOR ")

    origin_im_size = frame.shape[:-1]
    for obj in objects:
        # Validation bbox of detected object
        if obj['xmax'] > origin_im_size[1] or obj['ymax'] > origin_im_size[0] or obj['xmin'] < 0 or obj['ymin'] < 0:
            continue
        color = (int(min(obj['class_id'] * 12.5, 255)),
                 min(obj['class_id'] * 7, 255), min(obj['class_id'] * 5, 255))
        det_label = labels_map[obj['class_id']] if labels_map and len(labels_map) >= obj['class_id'] else \
            str(obj['class_id'])

        if args.raw_output_message:
            log.info(
                "{:^9} | {:10f} | {:4} | {:4} | {:4} | {:4} | {} ".format(det_label, obj['confidence'], obj['xmin'],
                                                                          obj['ymin'], obj['xmax'], obj['ymax'],
                                                                          color))

        cv2.rectangle(frame, (obj['xmin'], obj['ymin']), (obj['xmax'], obj['ymax']), color, 2)
        cv2.putText(frame,
                    "#" + det_label + ' ' + str(round(obj['confidence'] * 100, 1)) + ' %',
                    (obj['xmin'], obj['ymin'] - 7), cv2.FONT_HERSHEY_COMPLEX, 0.6, color, 1)
        cv2.imshow("DetectionResults", frame)

        key = cv2.waitKey(wait_key_code)

        # ESC key
        if key == 27:
            break

    cv2.destroyAllWindows()

AttributeError: 'openvino.inference_engine.ie_api.ExecutableNetwork' object has no attribute 'layers'

In [30]:
cv2.destroyAllWindows()

In [31]:
cap.release()

In [32]:
#%matplotlib inline

import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Dropout, Convolution2D, MaxPooling2D, Flatten
from keras.optimizers import adam
from keras.utils.np_utils import to_categorical
from keras import backend as K
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
# input image dimensions
img_rows, img_cols = 28, 28

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train/255
x_test = x_test/255

y_dummy_train = to_categorical(y_train)
y_dummy_test = to_categorical(y_test)


model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape = input_shape, activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(32, 3, 3, activation="relu"))
model.add(Convolution2D(32, 3, 3, activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer = "adam", metrics = ["accuracy"])


hist = model.fit(x_train, y_dummy_train,
                 validation_data = (x_test, y_dummy_test), nb_epoch=15, batch_size=64)

model.save("full_model.mnist")

[ WARNING ]  From %s: The name %s is deprecated. Please use %s instead.



C:\Program Files (x86)\IntelSWTools\openvino\deployment_tools\model_optimizer\mo_tf.py:38: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(28, 28, 1..., activation="relu")`
[ WARNING ]  From %s: The name %s is deprecated. Please use %s instead.



[ WARNING ]  From %s: The name %s is deprecated. Please use %s instead.



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


C:\Program Files (x86)\IntelSWTools\openvino\deployment_tools\model_optimizer\mo_tf.py:41: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
C:\Program Files (x86)\IntelSWTools\openvino\deployment_tools\model_optimizer\mo_tf.py:42: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
[ WARNING ]  From %s: calling %s (from %s) with %s is deprecated and will be removed %s.
Instructions for updating:
%s


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 32)          9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               65664     
__________

[ WARNING ]  From %s: The name %s is deprecated. Please use %s instead.

C:\Program Files (x86)\IntelSWTools\openvino\deployment_tools\model_optimizer\mo_tf.py:56: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


[ WARNING ]  From %s: %s (from %s) is deprecated and will be removed %s.
Instructions for updating:
%s


Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 23s 386us/step - loss: 0.2634 - acc: 0.9170 - val_loss: 0.0623 - val_acc: 0.9804
Epoch 2/15
60000/60000 [==============================] - 21s 351us/step - loss: 0.0914 - acc: 0.9730 - val_loss: 0.0386 - val_acc: 0.9870
Epoch 3/15
60000/60000 [==============================] - 20s 334us/step - loss: 0.0662 - acc: 0.9803 - val_loss: 0.0291 - val_acc: 0.9899
Epoch 4/15
60000/60000 [==============================] - 20s 331us/step - loss: 0.0523 - acc: 0.9841 - val_loss: 0.0272 - val_acc: 0.9913
Epoch 5/15
60000/60000 [==============================] - 21s 355us/step - loss: 0.0442 - acc: 0.9861 - val_loss: 0.0274 - val_acc: 0.9909
Epoch 6/15
60000/60000 [==============================] - 19s 322us/step - loss: 0.0371 - acc: 0.9884 - val_loss: 0.0259 - val_acc: 0.9925
Epoch 7/15
60000/60000 [==============================] - 20s 331us/step - loss: 0.0331 - acc: 0.9897 - val_loss: 0.

In [36]:
from keras import backend as K
# This line must be executed before loading Keras model.
K.set_learning_phase(0)

In [37]:
from keras.models import load_model
model = load_model('mnist.h5')
print(model.outputs)
# [<tf.Tensor 'dense_2/Softmax:0' shape=(?, 10) dtype=float32>]
print(model.inputs)
# [<tf.Tensor 'conv2d_1_input:0' shape=(?, 28, 28, 1) dtype=float32>]

[<tf.Tensor 'dense_2_1/Softmax:0' shape=(?, 10) dtype=float32>]
[<tf.Tensor 'conv2d_1_input_1:0' shape=(?, 28, 28, 1) dtype=float32>]


In [38]:
from keras import backend as K
import tensorflow as tf

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph


frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

INFO:tensorflow:Froze 72 variables.


[ INFO ]  Froze %d variables.


INFO:tensorflow:Converted 72 variables to const ops.


[ INFO ]  Converted %d variables to const ops.


In [39]:
# Save to ./model/tf_model.pb
tf.train.write_graph(frozen_graph, "model", "tf_model.pb", as_text=False)

'model\\tf_model.pb'